In [21]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import re
import os
import pickle
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from scipy import sparse 
from scipy import stats
from statsmodels.stats.proportion import proportion_confint
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'capstone_user_identification'

In [3]:
def to_csr_matrix(X_matrix):
    
    # Конвертируем матрицу частот в разреженую матрицу по принципу BagOfWord
    # формируем координаты для данных
    #data = np.ones(len(X_matrix.flatten()), dtype=np.int)
    
    data = (X_matrix.flatten() != 0).astype(int)[X_matrix.flatten() != 0]
    col = X_matrix.flatten()[X_matrix.flatten() != 0]
    row = np.array([[n] * len(X_matrix[0]) for n in range(len(X_matrix))]).flatten()[X_matrix.flatten() != 0]
    
    X_sparse = csr_matrix((data, (row, col-1)), shape=(len(X_matrix), X_matrix.max()))
    return X_sparse
    

In [119]:
def prepare_sparse_train_set_window(path_to_csv_files, site_freq_path, 
                                    session_length=10, window_size=10):
    ''' ВАШ КОД ЗДЕСЬ'''
    files = glob(os.path.join(path_to_csv_files, '*.csv'))
    
    num_filter = r'[0-9]{4}' #Фильтр для user_id
    
    matrix = False
    
    with open(site_freq_path, 'rb') as handle:
        site_freq = pickle.load(handle)
    site_freq_len = len(site_freq)
    
    for filename in files:
        #Собираем данные по сессиям 
        df = pd.read_csv(filename, parse_dates=True)
        user_id = int(re.findall(num_filter, filename)[-1])
        
        #Заменим названия сайтов на их ID
        df['site'] = df['site'].map(site_freq).apply(lambda x: x[0])
        
        #Формирование sparse matrix для пользователя user_id
        data_sites = [] #Разметка для sparse matrix по индексам сайтов
        data_sessions = [] #Разметка для sparse matrix по индексам сессий
        session_id = 0 #Счетчик сессий

        for i in range(0, len(df), window_size): #Цикл с шагом Window_size по загруженному DF, генерируем сессиии заданной длительности и с окном сдвига
            
            data_ = df.iloc[i:i + session_length, 1].values.tolist() #Выбираем session_length данных и конвертируем в list, даже если столько данных нет - работает
            
            data_sites.extend(data_) #Записываем их в list сайтов
            data_sessions.extend([session_id for _ in range(len(data_))]) #Генерируем list сессий равный количеству сайтов в сессии
            session_id += 1
        
        #Собираем матрицу и вектор ответов
        if sparse.issparse(matrix) == False:
            matrix = sparse.csr_matrix((np.array([1] * len(data_sessions)),
                                       (np.array(data_sessions), np.array(data_sites)-1)),
                                       shape=(session_id+1, site_freq_len))
            users = np.array([user_id] * (session_id+1)).reshape((session_id+1, 1))
        else:
            matrix = sparse.vstack([matrix, sparse.csr_matrix((np.array([1] * len(data_sessions)),
                                       (np.array(data_sessions), np.array(data_sites)-1)),
                                       shape=(session_id+1, site_freq_len))])
            users = np.vstack([users, np.array([user_id] * (session_id+1)).reshape((session_id+1, 1))])
        
        
        
    return matrix, users
    


In [137]:
X_toy_s5_w3, y_s5_w3 = prepare_sparse_train_set_window(os.path.join(PATH_TO_DATA,'3users'), 
                                                       os.path.join(PATH_TO_DATA,'site_freq_3users.pkl'),
                                                       session_length=5, window_size=3)

In [138]:
X_toy_s5_w3.todense()

matrix([[0, 0, 0, 1, 0, 0, 0, 3, 0, 1, 0],
        [1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0],
        [1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 3, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 2, 0, 1, 2, 0, 0, 0],
        [0, 0, 0, 0, 3, 1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [129]:
session_length=5
window_size=5
path_to_csv_files = os.path.join(PATH_TO_DATA,'150users')
site_freq_path = os.path.join(PATH_TO_DATA,'site_freq_150users.pkl')

files = glob(os.path.join(path_to_csv_files, '*.csv'))
    
num_filter = r'[0-9]{4}' #Фильтр для user_id

matrix = False

with open(site_freq_path, 'rb') as handle:
    site_freq = pickle.load(handle)
site_freq_len = len(site_freq)

for filename in files:
    #Собираем данные по сессиям 
    df = pd.read_csv(filename, parse_dates=True)
    user_id = int(re.findall(num_filter, filename)[-1])

    #Заменим названия сайтов на их ID
    df['site'] = df['site'].map(site_freq).apply(lambda x: x[0])

    #Формирование sparse matrix для пользователя user_id
    data_sites = [] #Разметка для sparse matrix по индексам сайтов
    data_sessions = [] #Разметка для sparse matrix по индексам сессий
    session_id = 0 #Счетчик сессий

    for i in range(0, len(df), window_size): #Цикл с шагом Window_size по загруженному DF, генерируем сессиии заданной длительности и с окном сдвига

        data_ = df.iloc[i:i + session_length, 1].values.tolist() #Выбираем session_length данных и конвертируем в list, даже если столько данных нет - работает

        data_sites.extend(data_) #Записываем их в list сайтов
        data_sessions.extend([session_id for _ in range(len(data_))]) #Генерируем list сессий равный количеству сайтов в сессии
        session_id += 1

    #Собираем матрицу и вектор ответов
    if sparse.issparse(matrix) == False:
        matrix = sparse.csr_matrix((np.array([1] * len(data_sessions)),
                                   (np.array(data_sessions), np.array(data_sites)-1)),
                                   shape=(session_id+1, site_freq_len))
        users = np.array([user_id] * (session_id+1)).reshape((session_id+1, 1))
    else:
        matrix = sparse.vstack([matrix, sparse.csr_matrix((np.array([1] * len(data_sessions)),
                                   (np.array(data_sessions), np.array(data_sites)-1)),
                                   shape=(session_id+1, site_freq_len))])
        users = np.vstack([users, np.array([user_id] * (session_id+1)).reshape((session_id+1, 1))])

TypeError: 'DataFrame' object is not callable

In [133]:
df['site'].map(site_freq)#.apply(lambda x: x[0])

TypeError: 'DataFrame' object is not callable

In [135]:
df['site']

0              www.google.fr
1              www.google.fr
2            apis.google.com
3             www.google.com
4              www.google.fr
                ...         
21360       www.facebook.com
21361     i3.mangareader.net
21362    www.mangareader.net
21363       www.facebook.com
21364    www.mangareader.net
Name: site, Length: 21365, dtype: object

In [136]:
site_freq

,site,user_id
0,0.0-02.net,2
1,0.56.web1.im.weibo.com,5
2,0.academia-assets.com,13
3,0.docs.google.com,1147
4,0.drive.google.com,2322
...,...,...
27792,zupimages.net,23
27793,zurich.cloudsigma.cedexis.com,15
27794,zvon.org,56
27795,zx.findart.com.cn,7


In [87]:
sparse.vstack([[], second_matrix])[1:, :].sum(axis=1)

TypeError: 'coo_matrix' object is not subscriptable

In [73]:
first_matrix.sum() + second_matrix.sum()

21710

In [143]:
%%time
import itertools
from time import sleep
import tqdm

for num_users in tqdm_notebook([10, 150]):
    for window_size, session_length in tqdm_notebook(itertools.product([10, 7, 5], [15, 10, 7, 5])):
        if window_size <= session_length and (window_size, session_length) != (10, 10):
            sleep(.1)
                 



Wall time: 1.76 s
